In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
from ydata_profiling import ProfileReport

/Users/muskan/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Define the list of columns to be read from the CSV file
rate_fields = ['StateCode','PlanId','IndividualRate','PrimarySubscriberAndThreeOrMoreDependents','BusinessYear','Age','RatingAreaId','Tobacco','SourceName']
# Read the CSV in chunks for efficient memory usage
rate_chunks = pd.read_csv("/Users/muskan/Documents/SOEN 6111/Rate.csv",iterator=True, chunksize=1000, usecols=rate_fields)
# Concatenating all chunks into a single DataFrame
rates_concat = pd.concat(chunk for chunk in rate_chunks)

In [3]:
# Display the first 5 rows to check the data
rates_concat.head(5)

,BusinessYear,StateCode,SourceName,PlanId,RatingAreaId,Tobacco,Age,IndividualRate,PrimarySubscriberAndThreeOrMoreDependents
0,2014,AK,HIOS,21989AK0010001,Rating Area 1,No Preference,0-20,29.00,NaN
1,2014,AK,HIOS,21989AK0020001,Rating Area 1,No Preference,Family Option,36.95,107.61
2,2014,AK,HIOS,21989AK0020001,Rating Area 2,No Preference,Family Option,36.95,107.61
3,2014,AK,HIOS,21989AK0010001,Rating Area 1,No Preference,21,32.00,NaN
4,2014,AK,HIOS,21989AK0010001,Rating Area 1,No Preference,22,32.00,NaN


In [4]:
# Listing column names of the DataFrame
rates_concat.columns

Index(['BusinessYear', 'StateCode', 'SourceName', 'PlanId', 'RatingAreaId',
       'Tobacco', 'Age', 'IndividualRate',
       'PrimarySubscriberAndThreeOrMoreDependents'],
      dtype='object')

In [5]:
# Prepare the DataFrame for further manipulation
df = rates_concat
# Create a new column 'tobacco_rate'. If 'Tobacco' column is not 'Tobacco User/Non-Tobacco User', copy 'IndividualRate';
# otherwise, increase 'IndividualRate' by 3% to simulate a tobacco surcharge.
df['tobacco_rate'] = np.where(df['Tobacco']!= 'Tobacco User/Non-Tobacco User ',df['IndividualRate'],df['IndividualRate']*1.03)
# Display the DataFrame to verify changes
df.head()

,BusinessYear,StateCode,SourceName,PlanId,RatingAreaId,Tobacco,Age,IndividualRate,PrimarySubscriberAndThreeOrMoreDependents,tobacco_rate
0,2014,AK,HIOS,21989AK0010001,Rating Area 1,No Preference,0-20,29.00,NaN,29.00
1,2014,AK,HIOS,21989AK0020001,Rating Area 1,No Preference,Family Option,36.95,107.61,36.95
2,2014,AK,HIOS,21989AK0020001,Rating Area 2,No Preference,Family Option,36.95,107.61,36.95
3,2014,AK,HIOS,21989AK0010001,Rating Area 1,No Preference,21,32.00,NaN,32.00
4,2014,AK,HIOS,21989AK0010001,Rating Area 1,No Preference,22,32.00,NaN,32.00


In [6]:
# Data cleaning for 'Age' column:
# Assign -5 to 'Family Option' to mark family plans distinctively.
# Replace '0-20' with 10 to represent minors as a single group.
# Assign 100 to '65 and over' to distinctly represent seniors.
df['Age'] = np.where(df['Age']== 'Family Option',-5,df['Age']) # Family Plan = -5
df['Age'] = np.where(df['Age']== '0-20',10,df['Age'])  # Minor = 10 below 20 years of age
df['Age'] = np.where(df['Age']== '65 and over',100,df['Age'])  # Old age = 100 more than 65 years of age


In [7]:
# Display the DataFrame to verify changes
df.head()

,BusinessYear,StateCode,SourceName,PlanId,RatingAreaId,Tobacco,Age,IndividualRate,PrimarySubscriberAndThreeOrMoreDependents,tobacco_rate
0,2014,AK,HIOS,21989AK0010001,Rating Area 1,No Preference,10,29.00,NaN,29.00
1,2014,AK,HIOS,21989AK0020001,Rating Area 1,No Preference,-5,36.95,107.61,36.95
2,2014,AK,HIOS,21989AK0020001,Rating Area 2,No Preference,-5,36.95,107.61,36.95
3,2014,AK,HIOS,21989AK0010001,Rating Area 1,No Preference,21,32.00,NaN,32.00
4,2014,AK,HIOS,21989AK0010001,Rating Area 1,No Preference,22,32.00,NaN,32.00


In [8]:
# Display the shape of the DataFrame to understand its size
df.shape

(12694445, 10)

In [9]:
# Filling missing values in 'PrimarySubscriberAndThreeOrMoreDependents' column with 0
df['PrimarySubscriberAndThreeOrMoreDependents']=df['PrimarySubscriberAndThreeOrMoreDependents'].fillna(0) 

In [10]:
# Updating 'tobacco_rate' based on 'PrimarySubscriberAndThreeOrMoreDependents' column values
# Adjust 'tobacco_rate' for entries with dependents, using the value in 'PrimarySubscriberAndThreeOrMoreDependents' if greater than 0.
df['tobacco_rate'] = np.where(df['PrimarySubscriberAndThreeOrMoreDependents'] > 0,df['PrimarySubscriberAndThreeOrMoreDependents'], df['tobacco_rate'])
# Updating 'PrimarySubscriberAndThreeOrMoreDependents' column values
# Normalize 'PrimarySubscriberAndThreeOrMoreDependents' to binary values, 1 if there are dependents and 0 otherwise.
df['PrimarySubscriberAndThreeOrMoreDependents'] = np.where(df['PrimarySubscriberAndThreeOrMoreDependents'] > 0,1, df['PrimarySubscriberAndThreeOrMoreDependents'])
df

,BusinessYear,StateCode,SourceName,PlanId,RatingAreaId,Tobacco,Age,IndividualRate,PrimarySubscriberAndThreeOrMoreDependents,tobacco_rate
0,2014,AK,HIOS,21989AK0010001,Rating Area 1,No Preference,10,29.00,0.0,29.00
1,2014,AK,HIOS,21989AK0020001,Rating Area 1,No Preference,-5,36.95,1.0,107.61
2,2014,AK,HIOS,21989AK0020001,Rating Area 2,No Preference,-5,36.95,1.0,107.61
3,2014,AK,HIOS,21989AK0010001,Rating Area 1,No Preference,21,32.00,0.0,32.00
4,2014,AK,HIOS,21989AK0010001,Rating Area 1,No Preference,22,32.00,0.0,32.00
...,...,...,...,...,...,...,...,...,...,...
12694440,2016,WV,SERFF,96480WV0090003,Rating Area 11,No Preference,61,14.05,0.0,14.05
12694441,2016,WV,SERFF,96480WV0090003,Rating Area 11,No Preference,62,14.05,0.0,14.05
12694442,2016,WV,SERFF,96480WV0090003,Rating Area 11,No Preference,63,14.05,0.0,14.05
12694443,2016,WV,SERFF,96480WV0090003,Rating Area 11,No Preference,64,14.05,0.0,14.05


In [11]:
# Convert 'Age' to integer type for consistency in data types.
df.Age = df.Age.astype(int)

In [12]:
# Generating a Profile Report for the DataFrame
# Generate a profile report for exploratory data analysis, capturing insights about the dataset.
profile_final = ProfileReport(df, title="Profiling Report Rates CSV", explorative=True)

# Saving the Profile Report as an HTML file
profile_final.to_file("Rate_Cleaned_Output.html")

Summarize dataset:  71%|███████   | 12/17 [01:27<00:19,  3.96s/it, Calculate auto correlation]                                /Users/muskan/Library/Python/3.9/lib/python/site-packages/ydata_profiling/model/correlations.py:66: UserWarning: There was an attempt to calculate the auto correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"auto": {"calculate": False}})`
If this is problematic for your use case, please report this as an issue:
https://github.com/ydataai/ydata-profiling/issues
(include the error message: 'could not convert string to float: 'HIOS'')
  warnings.warn(
Export report to file: 100%|██████████| 1/1 [00:00<00:00, 350.99it/s]


In [13]:

# Filtering rows where 'Tobacco' column is 'Tobacco User/Non-Tobacco User'
# Filter rows where 'Tobacco' is 'Tobacco User/Non-Tobacco User' for further manipulation.
df_with_filters = df[df['Tobacco'] == 'Tobacco User/Non-Tobacco User']

# create a copy of the filtered dataframe
# Create copies of the filtered DataFrame to differentiate between tobacco users and non-users.
df_first_half = df_with_filters.copy()
df_second_half = df_with_filters.copy()

# change the value in the Tobacco column to 'Tobacco User' for the first half of the rows
# For the first copy, set 'Tobacco' to 'Tobacco User' for all rows to categorize these entries explicitly.
df_first_half.loc[df_first_half.index[:len(df_with_filters)], 'Tobacco'] = 'Tobacco User'

# change the value in the Tobacco column to 'Non-Tobacco User' for the second half of the rows
# For the second copy, change 'Tobacco' to 'Non-Tobacco User', creating a clear distinction.
df_second_half.loc[df_first_half.index[:len(df_with_filters)], 'Tobacco'] = 'Non-Tobacco User'

# Modifying 'tobacco_rate' column values for 'Non-Tobacco User' rows
# For 'Non-Tobacco User' rows, align 'tobacco_rate' with 'IndividualRate', effectively undoing the previous surcharge.
df_second_half.loc[df_first_half['Tobacco'] == 'Non-Tobacco User', 'tobacco_rate'] = df_second_half.loc[df_first_half['Tobacco'] == 'Non-Tobacco User', 'IndividualRate']

# concatenate the original dataframe with the modified filtered dataframe
# Combine the modified DataFrames with the rest of the dataset, reintegrating the separated tobacco user statuses.
df_concat = pd.concat([df_first_half,df_second_half])
df_processed = pd.concat([df[df['Tobacco'] != 'Tobacco User/Non-Tobacco User'], df_concat])

# reset the index
# Reset the index of the final DataFrame for consistency and ease of data manipulation.
df_processed = df_processed.reset_index(drop=True)

# Display the modified DataFrame for verification.
df_processed

,BusinessYear,StateCode,SourceName,PlanId,RatingAreaId,Tobacco,Age,IndividualRate,PrimarySubscriberAndThreeOrMoreDependents,tobacco_rate
0,2014,AK,HIOS,21989AK0010001,Rating Area 1,No Preference,10,29.00,0.0,29.00
1,2014,AK,HIOS,21989AK0020001,Rating Area 1,No Preference,-5,36.95,1.0,107.61
2,2014,AK,HIOS,21989AK0020001,Rating Area 2,No Preference,-5,36.95,1.0,107.61
3,2014,AK,HIOS,21989AK0010001,Rating Area 1,No Preference,21,32.00,0.0,32.00
4,2014,AK,HIOS,21989AK0010001,Rating Area 1,No Preference,22,32.00,0.0,32.00
...,...,...,...,...,...,...,...,...,...,...
17584562,2016,WV,SERFF,50328WV0020018,Rating Area 11,Non-Tobacco User,61,702.16,0.0,702.16
17584563,2016,WV,SERFF,50328WV0020018,Rating Area 11,Non-Tobacco User,62,717.91,0.0,717.91
17584564,2016,WV,SERFF,50328WV0020018,Rating Area 11,Non-Tobacco User,63,737.65,0.0,737.65
17584565,2016,WV,SERFF,50328WV0020018,Rating Area 11,Non-Tobacco User,64,749.64,0.0,749.64


In [14]:
# Save the cleaned and combined DataFrame to a CSV file for future use.
df_processed.to_csv('Rate_Cleaned.csv')

In [15]:
# Display the head of the final DataFrame to confirm the structure and changes.
df_processed.head()

,BusinessYear,StateCode,SourceName,PlanId,RatingAreaId,Tobacco,Age,IndividualRate,PrimarySubscriberAndThreeOrMoreDependents,tobacco_rate
0,2014,AK,HIOS,21989AK0010001,Rating Area 1,No Preference,10,29.00,0.0,29.00
1,2014,AK,HIOS,21989AK0020001,Rating Area 1,No Preference,-5,36.95,1.0,107.61
2,2014,AK,HIOS,21989AK0020001,Rating Area 2,No Preference,-5,36.95,1.0,107.61
3,2014,AK,HIOS,21989AK0010001,Rating Area 1,No Preference,21,32.00,0.0,32.00
4,2014,AK,HIOS,21989AK0010001,Rating Area 1,No Preference,22,32.00,0.0,32.00
